# FPL Draft League Analysis

This notebook analyzes the FPL Draft League data to show:
- Player picks by each manager
- League summary statistics
- Top available players
- Waiver order (if available)

## Setup and Data Loading

In [ ]:
import pandas as pd
import os
from pathlib import Path

# Set display options for better output
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
def load_draft_data():
    """Load the latest draft league data"""
    # Get the notebook's directory and navigate to data
    notebook_dir = Path().absolute()
    base_dir = notebook_dir.parent
    data_dir = base_dir / "data" / "draft_league" / "latest"
    
    try:
        # Load all data files
        managers_df = pd.read_csv(data_dir / "managers.csv")
        picks_df = pd.read_csv(data_dir / "picks.csv")
        players_df = pd.read_csv(data_dir / "players.csv")
        
        print(f"✅ Loaded data: {len(managers_df)} managers, {len(picks_df)} picks, {len(players_df)} players")
        return managers_df, picks_df, players_df
    except FileNotFoundError as e:
        print(f"❌ Error loading data: {e}")
        print("Make sure to run the draft data fetch script first!")
        return None, None, None

# Load the data
managers_df, picks_df, players_df = load_draft_data()

## Helper Functions

In [ ]:
def get_player_info(player_id, players_df):
    """Get player information from player ID"""
    player = players_df[players_df['id'] == player_id]
    if player.empty:
        return f"Unknown Player (ID: {player_id})"
    
    player = player.iloc[0]
    position_map = {1: "GKP", 2: "DEF", 3: "MID", 4: "FWD"}
    position = position_map.get(player['element_type'], "UNK")
    
    return {
        'name': player['web_name'],
        'full_name': f"{player['first_name']} {player['second_name']}",
        'position': position,
        'team': player['team'],
        'total_points': player.get('total_points', 0),
        'price': player.get('draft_rank', 0)
    }

def get_team_name(team_id, teams_data=None):
    """Get team name from team ID (simplified for now)"""
    # Common team mapping - could be enhanced with teams.csv
    team_map = {
        1: "ARS", 2: "AVL", 3: "BOU", 4: "BRE", 5: "BRI",
        6: "CHE", 7: "CRY", 8: "EVE", 9: "FUL", 10: "IPS",
        11: "LEI", 12: "LIV", 13: "MCI", 14: "MUN", 15: "NEW",
        16: "NFO", 17: "SOU", 18: "TOT", 19: "WHU", 20: "WOL"
    }
    return team_map.get(team_id, f"Team {team_id}")

## League Summary Statistics

Let's start with an overview of the league:

In [ ]:
def display_summary_stats(managers_df, picks_df, players_df):
    """Display summary statistics"""
    print("=" * 80)
    print("📈 LEAGUE SUMMARY")
    print("=" * 80)
    
    picked_players = picks_df[picks_df['owner'].notna()].copy()
    total_picked = len(picked_players)
    total_available = len(picks_df) - total_picked
    
    print(f"👥 Total Managers: {len(managers_df)}")
    print(f"⚽ Players Picked: {total_picked}")
    print(f"🆓 Players Available: {total_available}")
    print(f"📦 Total Player Pool: {len(picks_df)}")
    
    # Players per manager
    print(f"\n🔢 Players per Manager:")
    for _, manager in managers_df.iterrows():
        if pd.isna(manager['entry_id']) or manager['entry_id'] == '':
            manager_picks = pd.DataFrame()
        else:
            manager_entry_id = int(manager['entry_id'])
            picked_players['owner'] = picked_players['owner'].astype(int)
            manager_picks = picked_players[picked_players['owner'] == manager_entry_id]
        team_name = manager['entry_name']
        print(f"   {team_name}: {len(manager_picks)} players")

# Display summary
if managers_df is not None:
    display_summary_stats(managers_df, picks_df, players_df)

## Manager Squads

Let's see what players each manager has picked:

In [ ]:
def display_manager_squads(managers_df, picks_df, players_df):
    """Display each manager's squad"""
    print("=" * 80)
    print("🏆 FPL DRAFT LEAGUE - PLAYER PICKS BY MANAGER")
    print("=" * 80)
    
    # Get only picked players (those with an owner)
    picked_players = picks_df[picks_df['owner'].notna()].copy()
    # Convert owner to int for proper matching
    picked_players['owner'] = picked_players['owner'].astype(int)
    
    for _, manager in managers_df.iterrows():
        manager_id = manager['id']
        team_name = manager['entry_name']
        manager_name = f"{manager['player_first_name']} {manager['player_last_name']}"
        
        print(f"\n🔥 {team_name}")
        print(f"👤 Manager: {manager_name}")
        print("-" * 60)
        
        # Get this manager's picks (picks use entry_id, not id)
        if pd.isna(manager['entry_id']) or manager['entry_id'] == '':
            manager_picks = pd.DataFrame()  # Empty dataframe for managers without entry_id
        else:
            manager_entry_id = int(manager['entry_id'])
            manager_picks = picked_players[picked_players['owner'] == manager_entry_id]
        
        if manager_picks.empty:
            print("   No players picked yet")
            continue
        
        # Group players by position
        squad_by_position = {"GKP": [], "DEF": [], "MID": [], "FWD": []}
        
        for _, pick in manager_picks.iterrows():
            player_info = get_player_info(pick['element'], players_df)
            if isinstance(player_info, dict):
                squad_by_position[player_info['position']].append(player_info)
        
        # Display squad by position
        for position in ["GKP", "DEF", "MID", "FWD"]:
            if squad_by_position[position]:
                print(f"\n   {position}:")
                for player in sorted(squad_by_position[position], key=lambda x: x['name']):
                    team_name = get_team_name(player['team'])
                    points = player['total_points']
                    print(f"      • {player['name']} ({team_name}) - {points} pts")
        
        total_players = sum(len(squad_by_position[pos]) for pos in squad_by_position)
        print(f"\n   📊 Total Players: {total_players}")

# Display manager squads
if managers_df is not None:
    display_manager_squads(managers_df, picks_df, players_df)

## Top Available Players

Let's see who are the best available players to pick up:

In [ ]:
def display_top_available_players(picks_df, players_df, limit=15):
    """Display top available players"""
    print(f"=" * 80)
    print(f"🆓 TOP {limit} AVAILABLE PLAYERS")
    print("=" * 80)
    
    # Get unpicked players
    available_players = picks_df[picks_df['owner'].isna()].copy()
    
    if available_players.empty:
        print("   No players available (all drafted)")
        return
    
    # Get player details and sort by draft rank (lower is better)
    available_with_details = []
    for _, pick in available_players.iterrows():
        player_info = get_player_info(pick['element'], players_df)
        if isinstance(player_info, dict):
            available_with_details.append(player_info)
    
    # Sort by draft rank (lower rank = higher pick)
    top_available = sorted(available_with_details, key=lambda x: x['price'])[:limit]
    
    print(f"{'Player':<20} {'Team':<5} {'Pos':<4} {'Points':<7} {'Rank'}")
    print("-" * 50)
    
    for player in top_available:
        team_name = get_team_name(player['team'])
        print(f"{player['name']:<20} {team_name:<5} {player['position']:<4} {player['total_points']:<7} {player['price']}")

# Display top available players
if managers_df is not None:
    display_top_available_players(picks_df, players_df)

## Data Exploration

Let's explore the data in more detail with pandas:

In [ ]:
# Display basic info about our datasets
if managers_df is not None:
    print("Managers DataFrame:")
    print(managers_df.head())
    print(f"Shape: {managers_df.shape}")
    print(f"Columns: {list(managers_df.columns)}")

In [ ]:
if picks_df is not None:
    print("\nPicks DataFrame:")
    print(picks_df.head())
    print(f"Shape: {picks_df.shape}")
    print(f"Columns: {list(picks_df.columns)}")
    
    # Show picked vs available
    picked_count = picks_df['owner'].notna().sum()
    available_count = picks_df['owner'].isna().sum()
    print(f"\nPicked players: {picked_count}")
    print(f"Available players: {available_count}")

In [ ]:
if players_df is not None:
    print("\nPlayers DataFrame:")
    print(players_df.head())
    print(f"Shape: {players_df.shape}")
    print(f"Columns: {list(players_df.columns)}")
    
    # Show position distribution
    position_map = {1: "GKP", 2: "DEF", 3: "MID", 4: "FWD"}
    position_counts = players_df['element_type'].map(position_map).value_counts()
    print(f"\nPosition distribution:")
    print(position_counts)

## Advanced Analysis

Let's do some more advanced analysis:

In [ ]:
# Analyze the most popular teams for drafting
if picks_df is not None and players_df is not None:
    picked_players = picks_df[picks_df['owner'].notna()].copy()
    
    if not picked_players.empty:
        # Merge with player data to get team info
        picked_with_teams = picked_players.merge(players_df[['id', 'team']], left_on='element', right_on='id')
        
        # Count picks by team
        team_picks = picked_with_teams['team'].value_counts()
        
        print("🏅 Most Drafted Teams:")
        print("-" * 30)
        for team_id, count in team_picks.head(10).items():
            team_name = get_team_name(team_id)
            print(f"{team_name}: {count} players")

In [ ]:
# Show highest scoring available players
if picks_df is not None and players_df is not None:
    available_players = picks_df[picks_df['owner'].isna()].copy()
    
    if not available_players.empty:
        # Merge with player data
        available_with_stats = available_players.merge(players_df[['id', 'web_name', 'total_points', 'element_type', 'team']], 
                                                      left_on='element', right_on='id')
        
        # Sort by total points
        top_scorers = available_with_stats.nlargest(10, 'total_points')
        
        print("\n⭐ Highest Scoring Available Players:")
        print("-" * 50)
        position_map = {1: "GKP", 2: "DEF", 3: "MID", 4: "FWD"}
        
        for _, player in top_scorers.iterrows():
            position = position_map.get(player['element_type'], "UNK")
            team_name = get_team_name(player['team'])
            print(f"{player['web_name']:<20} ({team_name}) {position:<4} - {player['total_points']} pts")

## Custom Analysis

You can use the loaded dataframes to do your own custom analysis:

In [ ]:
# Example: Find the manager with the most total points
if all(df is not None for df in [managers_df, picks_df, players_df]):
    picked_players = picks_df[picks_df['owner'].notna()].copy()
    picked_players['owner'] = picked_players['owner'].astype(int)
    
    # Merge with player stats
    picks_with_stats = picked_players.merge(players_df[['id', 'total_points']], 
                                           left_on='element', right_on='id')
    
    # Calculate total points per manager
    manager_points = picks_with_stats.groupby('owner')['total_points'].sum().reset_index()
    
    # Merge with manager names
    manager_points = manager_points.merge(managers_df[['entry_id', 'entry_name']], 
                                         left_on='owner', right_on='entry_id')
    
    # Sort by total points
    manager_points = manager_points.sort_values('total_points', ascending=False)
    
    print("🏆 Manager Rankings by Total Points:")
    print("-" * 40)
    for i, (_, manager) in enumerate(manager_points.iterrows(), 1):
        print(f"{i}. {manager['entry_name']}: {manager['total_points']} pts")

In [ ]:
# Your custom analysis here...
# Example ideas:
# - Position analysis (most picked positions)
# - Price vs performance analysis
# - Team diversity analysis
# - etc.